In [1]:
import numpy as np
import pandas as pd
import os
import csv
import re
from dotenv import load_dotenv
import json
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import glob
import PyPDF2 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from PyPDF2 import PdfReader
import requests

/home/cptaswadu/RESCUE-n8n/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = '/home/cptaswadu/RESCUE-n8n/insurance'
load_dotenv(dotenv_path=os.path.join(path, ".env"))
openai_api_key = os.getenv("OPEN_AI_API_KEY")
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [3]:
class RAGPolicyRetriever:
    def __init__(self, policy_folder_path, openai_api_key=None, perplexity_api_key=None, llm_model="openai"):
        self.policy_folder_path = policy_folder_path
        self.policies = {}
        self.embeddings = {}
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.llm_model = llm_model
        self.openai_client = OpenAI(api_key=openai_api_key) if openai_api_key else None
        self.perplexity_api_key = perplexity_api_key

    def load_policies(self):
        pdf_files = glob.glob(os.path.join(self.policy_folder_path, "*.pdf"))
        for pdf_file in pdf_files:
            with open(pdf_file, "rb") as f:
                reader = PyPDF2.PdfReader(f)
                text = "".join(page.extract_text() or "" for page in reader.pages)
            self.policies[os.path.basename(pdf_file)] = text
        print(f"✅ Loaded {len(self.policies)} policies.")

    def embed_policies(self):
        for doc_name, doc_text in self.policies.items():
            self.embeddings[doc_name] = self.embedder.encode([doc_text])[0]
        print("✅ Embeddings created.")

    def extract_insurance_and_test(self, patient_info):
        prompt = f"""You are a clinical insurance assistant.

Given the patient information below, identify:
1. The insurance provider (e.g., UnitedHealthcare, Aetna, Cigna, etc.)
2. The type of genetic test (e.g., Whole Exome Sequencing, BRCA1/2, Panel, etc.)

Respond in JSON format with keys 'insurance' and 'test'.

PATIENT INFORMATION:
{patient_info}
"""

        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]

        if self.llm_model == "openai":
            response = self.openai_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0
            )
            output = response.choices[0].message.content.strip()

        elif self.llm_model == "perplexity":
            headers = {
                "Authorization": f"Bearer {self.perplexity_api_key}",
                "Content-Type": "application/json"
            }
            data = {
                "model": "sonar-pro",
                "messages": messages,
                "temperature": 0
            }
            url = "https://api.perplexity.ai/chat/completions"
            res = requests.post(url, headers=headers, json=data)
            output = res.json()["choices"][0]["message"]["content"].strip()

        else:
            raise ValueError("Unsupported LLM model")

        try:
            output_json = json.loads(re.sub(r"```json|```", "", output).strip())
            insurance = output_json.get("insurance", None)
            test_name = output_json.get("test", None)
            return insurance, test_name
        except Exception as e:
            print(f"❗ JSON parsing error in extract_insurance_and_test: {e}")
            return None, None

    def filter_policies_by_insurance(self, insurance_name):
        filtered = {}
        for doc_name, doc_text in self.policies.items():
            if insurance_name and insurance_name.replace(" ", "").lower() in doc_name.replace(" ", "").lower():
                filtered[doc_name] = doc_text
        return filtered

    def find_top_policies(self, patient_info, insurance_name, top_k=5):
        filtered_policies = self.filter_policies_by_insurance(insurance_name)
        if not filtered_policies:
            print("❗ No policies matched the insurance. Using all policies.")
            filtered_policies = self.policies

        query_embedding = self.embedder.encode([patient_info])[0]
        scored_policies = []
        for doc_name, doc_text in filtered_policies.items():
            doc_embedding = self.embeddings[doc_name]
            score = cosine_similarity([query_embedding], [doc_embedding])[0][0]
            scored_policies.append((doc_name, score, doc_text))

        scored_policies.sort(key=lambda x: x[1], reverse=True)
        return scored_policies[:top_k]

    def rerank_policies(self, patient_info, candidates):
        candidate_texts = [c[2][:500].replace("\n", " ") for c in candidates]

        prompt = f"""You are an expert insurance policy assistant.

You will be given patient information and a list of candidate insurance policies.
Please select which candidate policy best matches the patient's situation.

Patient Information:
{patient_info}

Candidate Policies:"""

        for idx, text in enumerate(candidate_texts, 1):
            prompt += f"\n\nPolicy {idx}:\n{text}"

        prompt += """

Please answer with only the number of the most appropriate policy.
Do not explain. Just output the number.

Answer:"""

        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]

        if self.llm_model == "openai":
            response = self.openai_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0
            )
            result = response.choices[0].message.content.strip()
        elif self.llm_model == "perplexity":
            headers = {
                "Authorization": f"Bearer {self.perplexity_api_key}",
                "Content-Type": "application/json"
            }
            data = {
                "model": "sonar-pro",
                "messages": messages,
                "temperature": 0
            }
            url = "https://api.perplexity.ai/chat/completions"
            res = requests.post(url, headers=headers, json=data)
            result = res.json()["choices"][0]["message"]["content"].strip()
        else:
            raise ValueError("Unsupported LLM model")

        match = re.search(r'(\d+)', result)
        selected_idx = int(match.group(1)) - 1 if match else 0
        return candidates[selected_idx]

    def find_policy(self, patient_info):
        insurance, test = self.extract_insurance_and_test(patient_info)
        print(f"📌 LLM-detected insurance: {insurance}, test: {test}")

        candidates = self.find_top_policies(patient_info, insurance)
        print("📋 Top candidates:")
        for doc_name, score, _ in candidates:
            print(f"- {doc_name}: {score:.4f}")

        best_policy = self.rerank_policies(patient_info, candidates)
        return best_policy[0], best_policy[2]


In [4]:
class QnAExecutor:
    def __init__(self, questions_list, llm_model="openai", openai_client=None, perplexity_api_key=None):
        self.questions_list = questions_list
        self.formatted_questions = self.format_questions()
        self.llm_model = llm_model
        self.openai_client = openai_client
        self.perplexity_api_key = perplexity_api_key

    def format_questions(self):
        formatted_questions = []
        for q in self.questions_list:
            question_line = f"{q['id']}. {q['question']}"
            if q.get("options") == ["Free text answer"]:
                question_line += "\n(Free text answer allowed.)"
            else:
                question_line += f"\nOptions: {', '.join(q['options'])}"
                if "additional_if_yes" in q:
                    question_line += f"\nIf you answer 'Yes', ALSO select from: {', '.join(q['additional_if_yes'])}"
                if "additional_if_no" in q:
                    question_line += f"\nIf you answer 'No', ALSO select from: {', '.join(q['additional_if_no'])}"
            formatted_questions.append(question_line)
        return "\n\n".join(formatted_questions)

    def clean_json_response(self, response_text):
        original = response_text.strip()

        # Step 0: Check for hallucinated greeting (Perplexity fallback)
        if "how can I assist you" in original.lower() or "insurance-related questions" in original.lower():
            raise ValueError("Perplexity returned generic assistant response instead of JSON.")

        # Step 1: Try direct parsing
        try:
            return json.loads(original)
        except json.JSONDecodeError:
            pass

        # Step 2: Remove code block wrappers
        cleaned = re.sub(r"^```(?:json)?\\s*|\\s*```$", "", original, flags=re.IGNORECASE).strip()
        try:
            return json.loads(cleaned)
        except json.JSONDecodeError:
            pass

        # Step 3: Try to extract the first {...} JSON-like block
        match = re.search(r"(\{[\s\S]*?\})", original)
        if match:
            try:
                return json.loads(match.group(1))
            except json.JSONDecodeError:
                pass

        raise ValueError("No valid JSON found in the response.")

    def run_qna(self, patient_info, policy_name, policy_text, case_id, retrieval_model, qna_model):
        prompt = f"""
You are a clinical insurance assistant specializing in genetic testing coverage policies.

You will be given:

1. Patient clinical information (including their insurance provider, plan type, and state of residence).
2. Official insurance policy document text (strictly use this policy content for insurance coverage decision making).

Instructions:

- Answer all questions strictly based on the insurance policy document provided.
- Do NOT refer to general guidelines or policies from other insurance providers.
- If policy document does not clearly specify rules, you MAY use patient's clinical information to infer answers carefully.
- Do NOT assume coverage criteria from other insurers or general clinical guidelines unless explicitly stated in the policy.
- Output answers in JSON format ONLY.

==== PATIENT INFORMATION ====
{patient_info}

==== INSURANCE POLICY DOCUMENT (from URL: {policy_name}) ====
{policy_text}

==== QUESTIONS ====
{self.formatted_questions}

Output your answers in JSON format only and include the policy_url at the end.
"""

        messages = [
            {"role": "system", "content": "You are a clinical insurance assistant."},
            {"role": "user", "content": prompt}
        ]

        if self.llm_model == "openai":
            response = self.openai_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0
            )
            result_content = response.choices[0].message.content.strip()

        elif self.llm_model == "perplexity":
            headers = {
                "Authorization": f"Bearer {self.perplexity_api_key}",
                "Content-Type": "application/json"
            }
            data = {
                "model": "sonar-pro",
                "messages": messages,
                "temperature": 0
            }
            url = "https://api.perplexity.ai/chat/completions"
            res = requests.post(url, headers=headers, json=data)
            result_content = res.json()["choices"][0]["message"]["content"].strip()

        else:
            raise ValueError("Unsupported LLM model")

        result_json = {}

        try:
            result_json = self.clean_json_response(result_content)

            final_result = {}
            for k, v in result_json.items():
                if k == "policy_url":
                    continue
                if "_selection" in k or "_details" in k:
                    base_key = k.replace("_selection", "").replace("_details", "")
                    final_result[f"{base_key}_followup"] = [v] if isinstance(v, str) else v
                else:
                    final_result[k] = v

            folder_name = f"{retrieval_model}_{qna_model}"
            save_dir = f"/home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/{folder_name}"
            os.makedirs(save_dir, exist_ok=True)
            filename = os.path.join(save_dir, f"{case_id}_qna_result.json")
            with open(filename, "w") as f:
                json.dump(final_result, f, indent=2)

            print(f"✅ QnA result saved to {filename}")

        except Exception as e:
            print("❗ JSON parsing error:", e)
            final_result = {
                "error": "JSON parsing failed",
                "raw_content": result_content
            }

        print("QnA Result JSON:", final_result)
        return final_result


In [5]:
questions_file_path = "/home/cptaswadu/RESCUE-n8n/insurance/dataset/Insurance_Genetic_Testing_QA.json"

with open(questions_file_path, "r") as f:
    questions_data = json.load(f)

questions_list = questions_data["questions"]

In [6]:
case_ex = [
    {
        "id": "Case1",
        "patient_info": "An 8-year-old boy with neurodevelopmental delay and seizures. A prior chromosomal microarray test was negative. Whole exome sequencing (WES) has been requested by the genetic counselor to investigate potential underlying genetic causes that may guide diagnosis and future treatment decisions. There is also a family history of neurodevelopmental disorders, as his older brother was diagnosed with autism spectrum disorder. The patient is covered by United Healthcare Choice Plus through a family plan and resides in New Jersey."
    },
    {
        "id": "Case2",
        "patient_info": "An 8-year-old boy with mild learning difficulties and no significant neurological symptoms. There is no family history of genetic conditions, and his prior chromosomal microarray test was negative. Whole exome sequencing (WES) has been requested by his primary care provider (PCP) to explore potential genetic factors as part of general health screening and educational planning. The patient is covered by United Healthcare Choice Plus through a family plan and resides in New Jersey."
    },
    {
        "id": "Case3",
        "patient_info": "A 35-year-old woman with a strong family history of breast and ovarian cancer. Her mother was diagnosed with breast cancer at age 42, and her maternal aunt had ovarian cancer in her 50s. The patient herself has no history of cancer but has dense breast tissue and is considered at increased risk. The genetic counselor has recommended BRCA1/BRCA2 testing to assess her hereditary cancer risk and guide risk-reducing management decisions, including potential prophylactic options. The patient is covered by Aetna Open Access Managed Choice Plan and resides in California."
    },
    {
        "id": "Case4",
        "patient_info": "A 28-year-old woman with no family history of breast or ovarian cancer. The patient requested BRCA1/BRCA2 genetic testing after reading about genetic risks online. There were no prior specialist consultations or referrals, and no other clinical risk factors have been identified. The test was ordered directly by her primary care physician at the patient's request. The patient is enrolled in Aetna Open Access Managed Choice Plan and lives in Texas."
    }
]

In [7]:
models = ["chatgpt", "perplexity"]
model_map = {"chatgpt": "openai", "perplexity": "perplexity"}
model_combinations = [(r, q) for r in models for q in models]
       

for retrieval_model, qna_model in model_combinations:
    print(f"\n🚀 Running: {retrieval_model}_{qna_model}")

    retriever = RAGPolicyRetriever(
        policy_folder_path="/home/cptaswadu/RESCUE-n8n/insurance/insurance_policy",
        openai_api_key=openai_api_key,
        perplexity_api_key=perplexity_api_key,
        llm_model=model_map[retrieval_model]
    )
    retriever.load_policies()
    retriever.embed_policies()

    executor = QnAExecutor(
        questions_list=questions_list,
        llm_model=model_map[qna_model],
        openai_client=retriever.openai_client,
        perplexity_api_key=perplexity_api_key
    )

    for case in case_ex:
        case_id = case["id"]
        patient_info = case["patient_info"]

        try:
            policy_name, policy_text = retriever.find_policy(patient_info)
            executor.run_qna(
                patient_info=patient_info,
                policy_name=policy_name,
                policy_text=policy_text,
                case_id=case_id,
                retrieval_model=retrieval_model,
                qna_model=qna_model  
            )
        except Exception as e:
            print(f"❌ Failed on {case_id} ({retrieval_model}_{qna_model}): {e}")


🚀 Running: chatgpt_chatgpt
✅ Loaded 585 policies.
✅ Embeddings created.
📌 LLM-detected insurance: UnitedHealthcare, test: Whole Exome Sequencing
📋 Top candidates:
- United Healthcare_whole-exome-and-whole-genome-sequencing.pdf: 0.4831
- United Healthcare_genetic-testing-neuromuscular-disorders.pdf: 0.4247
- United Healthcare_genetic-testing-hereditary-cancer.pdf: 0.4086
- United Healthcare_preimplantation-genetic-testing.pdf: 0.3903
- United Healthcare_genetic-testing-cardiac-disease.pdf: 0.3890
✅ QnA result saved to /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/chatgpt_chatgpt/Case1_qna_result.json
QnA Result JSON: {'Q0': 'Whole Exome Sequencing (WES)', 'Q1': 'Yes', 'Q2': 'Yes', 'Q3': 'Yes', 'Q4': 'Yes', 'Q4_followup': ['ACMG'], 'Q5': 'Yes', 'Q6': 'No', 'Q7': 'Yes', 'Q8': 'No', 'Q9': 'Yes', 'Q9_followup': ['Yes'], 'Q10': 'No', 'Q10_followup': ['Diagnostic'], 'Q11': 'Yes', 'Q12': 'Yes', 'Q13': 'Not specified', 'Q14': 'Yes', 'Q14_followup': ['81415'], 'Q15': 'No', 'Q16': 'Ye

In [8]:
def merge_qna_jsons_to_csv(folder_path, output_csv_path):
    all_data = []

    for file in os.listdir(folder_path):
        if file.endswith("_qna_result.json"):
            case_id = file.replace("_qna_result.json", "")
            json_path = os.path.join(folder_path, file)

            with open(json_path, "r") as f:
                try:
                    result = json.load(f)
                    flat_result = {"case_id": case_id}

                    for k, v in result.items():
                        if isinstance(v, list):
                            flat_result[k] = "; ".join(map(str, v))
                        else:
                            flat_result[k] = v

                    all_data.append(flat_result)
                except Exception as e:
                    print(f"❗ Failed to parse {file}: {e}")

    if all_data:
        df = pd.DataFrame(all_data)
        df.to_csv(output_csv_path, index=False)
        print(f"✅ Merged CSV saved to: {output_csv_path}")
    else:
        print(f"⚠️ No valid QnA result files found in: {folder_path}")

def merge_all_combinations_to_csv(base_dir):
    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)
        if os.path.isdir(folder_path):
            output_csv = os.path.join(base_dir, f"{folder_name}.csv")
            merge_qna_jsons_to_csv(folder_path, output_csv)


merge_all_combinations_to_csv(
    base_dir="/home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG"
)

✅ Merged CSV saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/chatgpt_perplexity.csv
✅ Merged CSV saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/chatgpt_chatgpt.csv
✅ Merged CSV saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/perplexity_chatgpt.csv
✅ Merged CSV saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/perplexity_perplexity.csv


In [9]:
ground_truth = {
    "Case1": {
        "Q0": "Whole Exome Sequencing (WES)",
        "Q1": "Yes",
        "Q2": "Yes",
        "Q3": "Yes",
        "Q4": "Yes",
        "Q4_followup": [
      "ACMG"
    ],
        "Q5": "Yes",
        "Q6": "No",
        "Q7": "Yes",
        "Q8": "No",
        "Q9": "Yes",
        "Q9_followup": [
      "Yes"
    ],
        "Q10": "No",
        "Q10_followup": [
      "Diagnostic"
    ],
        "Q11": "Yes",
        "Q12": "No",
        "Q13": "Yes",
        "Q14": "Yes",
        "Q14_followup": [
      "81415", "81416"
    ],
        "Q15": "No",
        "Q16": "Yes"
  },
  
    "Case2": {
        "Q0": "Whole Exome Sequencing (WES)",
        "Q1": "Yes",
        "Q2": "No",
        "Q3": "No",
        "Q4": "No",
        "Q5": "Yes",
        "Q6": "No",
        "Q7": "No",
        "Q8": "No",
        "Q9": "Yes",
        "Q9_followup": [
      "No"
    ],
        "Q10": "No",
        "Q10_followup": [
      "Other"
    ],
        "Q11": "Yes",
        "Q12": "No",
        "Q13": "Yes",
        "Q14": "Yes",
        "Q14_followup": [
      "81415", "81416"
    ],
        "Q15": "No",
        "Q16": "No"
  },
  
    "Case3": {
        "Q0": "BRCA1/BRCA2 genetic testing",
        "Q1": "Yes",
        "Q2": "Yes",
        "Q3": "Yes",
        "Q4": "Yes",
        "Q4_followup": [
      "NCCN"
    ],
        "Q5": "Yes",
        "Q6": "No",
        "Q7": "No",
        "Q8": "No",
        "Q9": "Yes",
        "Q9_followup": [
      "Yes"
    ],
        "Q10": "No",
        "Q10_followup": [
      "Risk Assessment"
    ],
        "Q11": "Yes",
        "Q12": "No",
        "Q13": "Yes",
        "Q14": "Yes",
        "Q14_followup": [
      "81162"
    ],
        "Q15": "No",
        "Q16": "Yes"
  },
  
    "Case4": {
        "Q0": "BRCA1/BRCA2 genetic testing",
        "Q1": "Yes",
        "Q2": "Yes",
        "Q3": "No",
        "Q4": "No",
        "Q5": "No",
        "Q6": "No",
        "Q7": "No",
        "Q8": "No",
        "Q9": "Yes",
        "Q9_followup": [
      "No"
    ],
        "Q10": "No",
        "Q10_followup": [
      "Risk Assessment"
    ],
        "Q11": "No",
        "Q12": "No",
        "Q13": "Yes",
        "Q14": "Yes",
        "Q14_followup": [
      "81162"
    ],
        "Q15": "No",
        "Q16": "No"
  }
    
}

In [10]:
def load_converted_results_from_folder(folder_path):
    converted_results = {}
    for file in os.listdir(folder_path):
        if file.endswith("_qna_result.json"):
            case_id = file.replace("_qna_result.json", "")
            file_path = os.path.join(folder_path, file)
            with open(file_path, "r") as f:
                try:
                    data = json.load(f)
                except json.JSONDecodeError as e:
                    print(f"❗ JSON decode error in {file}: {e}")
                    continue

                clean = {}
                for k, v in data.items():
                    if k.endswith("_followup"):
                        clean[k] = v if isinstance(v, list) else [v]
                    else:
                        clean[k] = v

                converted_results[case_id] = clean
    return converted_results

In [11]:
def evaluate_qna_result(case_id, predicted_result, gold_result, folder_path=None):
    records = []
    correct_count = 0
    total_count = 0

    for qid in gold_result:
        if not qid.startswith("Q") or qid == "policy_url" or qid == "Q17" or "_followup" in qid:
            continue

        pred_answer = predicted_result.get(qid, "")
        gold_answer = gold_result.get(qid, "")

        if isinstance(pred_answer, list):
            pred_answer = ", ".join(pred_answer)
        if isinstance(gold_answer, list):
            gold_answer = ", ".join(gold_answer)

        pred_answer = pred_answer.strip()
        gold_answer = gold_answer.strip()

        is_correct = pred_answer == gold_answer
        score = 1 if is_correct else 0

        records.append({
            "Case": case_id,
            "Question": qid,
            "Predicted": pred_answer,
            "Gold": gold_answer,
            "Score": score
        })

        total_count += 1
        correct_count += score

        followup_key = qid + "_followup"
        pred_followup = predicted_result.get(followup_key, None)
        gold_followup = gold_result.get(followup_key, None)

        if is_correct and gold_followup is not None:
            def normalize(ans):
                if ans is None:
                    return "None"
                if isinstance(ans, list):
                    return ", ".join(ans)
                return ans

            pred_followup_norm = normalize(pred_followup)
            gold_followup_norm = normalize(gold_followup)

            followup_score = 1 if pred_followup_norm == gold_followup_norm else 0

            records.append({
                "Case": case_id,
                "Question": followup_key,
                "Predicted": pred_followup_norm,
                "Gold": gold_followup_norm,
                "Score": followup_score
            })

            total_count += 1
            correct_count += followup_score

    accuracy = correct_count / total_count * 100 if total_count > 0 else 0

    records.append({
        "Case": case_id,
        "Question": "TOTAL",
        "Predicted": f"Correct: {correct_count}",
        "Gold": f"Incorrect: {total_count - correct_count}",
        "Score": f"Accuracy: {accuracy:.2f}%"
    })

    df = pd.DataFrame(records)

    # ✅ Save CSV if folder_path is given
    if folder_path:
        os.makedirs(folder_path, exist_ok=True)
        csv_path = os.path.join(folder_path, f"evaluation_{case_id}.csv")
        df.to_csv(csv_path, index=False)
        print(f"📄 Saved evaluation to {csv_path}")

    return df, accuracy

    return pd.DataFrame(records), accuracy

In [12]:
def evaluate_all_folders_with_summary(base_dir, gold_answers, summary_output_csv):
    eval_output_dir = os.path.join(base_dir, "Evaluation")
    os.makedirs(eval_output_dir, exist_ok=True)

    summary_records = []

    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)
        if not os.path.isdir(folder_path):
            continue

        print(f"\n📊 Evaluating folder: {folder_name}")

        converted_results = load_converted_results_from_folder(folder_path)
        all_dfs = []
        accuracies = []

        for case_id, pred_result in converted_results.items():
            gold_result = gold_answers.get(case_id)
            if gold_result is None:
                continue

            df_case, acc = evaluate_qna_result(case_id, pred_result, gold_result)
            all_dfs.append(df_case)
            accuracies.append(acc)

        if all_dfs:
            merged_df = pd.concat(all_dfs, ignore_index=True)
            eval_csv_path = os.path.join(folder_path, "evaluation_results.csv")  # ⬅️ Save inside each folder
            merged_df.to_csv(eval_csv_path, index=False)
            print(f"✅ Saved: {eval_csv_path}")

            summary_records.append({
                "Model_Combination": folder_name,
                "Mean_Accuracy": f"{sum(accuracies) / len(accuracies):.2f}%" if accuracies else "N/A"
            })

    if summary_records:
        summary_df = pd.DataFrame(summary_records)
        os.makedirs(os.path.dirname(summary_output_csv), exist_ok=True)
        summary_df.to_csv(summary_output_csv, index=False)
        print(f"\n✅ Summary saved to: {summary_output_csv}")
        print(summary_df)


In [13]:
base_dir = "/home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG"
summary_output_csv = os.path.join(base_dir, "Evaluation", "summary_accuracy.csv")

evaluate_all_folders_with_summary(
    base_dir=base_dir,
    gold_answers=ground_truth,
    summary_output_csv=summary_output_csv
)


📊 Evaluating folder: chatgpt_perplexity
✅ Saved: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/chatgpt_perplexity/evaluation_results.csv

📊 Evaluating folder: Evaluation

📊 Evaluating folder: chatgpt_chatgpt
✅ Saved: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/chatgpt_chatgpt/evaluation_results.csv

📊 Evaluating folder: perplexity_chatgpt
✅ Saved: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/perplexity_chatgpt/evaluation_results.csv

📊 Evaluating folder: perplexity_perplexity
✅ Saved: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/perplexity_perplexity/evaluation_results.csv

✅ Summary saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/Evaluation/summary_accuracy.csv
       Model_Combination Mean_Accuracy
0     chatgpt_perplexity        64.64%
1        chatgpt_chatgpt        76.75%
2     perplexity_chatgpt        63.87%
3  perplexity_perplexity        65.56%
